In [ ]:
%load_ext autoreload
%autoreload 2 
#default_exp utils

In [ ]:
#exporti
from fastai.tabular.all import * 
from tabnet.model import * 

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Model creating functions

### classifier

In [ ]:
#export
@delegates(TabNetBase.__init__)
def TabNetClassifier(head_func, to, **kwargs):
    return TabNet(head_func, emb_szs=get_emb_sz(to), n_cont=len(to.cont_names), n_out=to.c, **kwargs)

### self supervised

In [ ]:
#export
@delegates(TabNetBase.__init__)
def TabNetSelfSupervised(head_func, to, bs=1024, **kwargs):
    n_out = len(get_emb_sz(to)) + len(to.cont_names)
    return TabNet(head_func, emb_szs=get_emb_sz(to), n_cont=len(to.cont_names), n_out=n_out, **kwargs)

# Self Supervised Data Loader 

In [ ]:
#exporti
def _maybe_expand(o): return o[:,None] if o.ndim==1 else o

In [ ]:
#export
class ReadTabBatchIdentity(ItemTransform):
    
    def __init__(self, to): store_attr()
        
    def encodes(self, to):
        if not to.with_cont: res = (tensor(to.cats).long(),)
        else: res = (tensor(to.cats).long(),tensor(to.conts).float())
        res = res + res #
        if to.device is not None: res = to_device(res, to.device)
        return res 
    
    
    def decodes(self, o):
        o = o[0:2]
        o = [_maybe_expand(o_) for o_ in to_np(o) if o_.size != 0]
        vals = np.concatenate(o, axis=1)
        try: df = pd.DataFrame(vals, columns=self.to.all_col_names)
        except: df = pd.DataFrame(vals, columns=self.to.x_names)
        to = self.to.new(df)
        return to

In [ ]:
#export
class TabularPandasIdentity(TabularPandas): pass 

In [ ]:
#export
@delegates()
class TabDataLoaderIdentity(TabDataLoader):
    do_item = noops
    def __init__(self, dataset, bs=16, shuffle=False, after_batch=None, num_workers=0, **kwargs):
        if after_batch is None: after_batch = L(TransformBlock().batch_tfms)+ReadTabBatchIdentity(dataset)
        super().__init__(dataset, bs=bs, shuffle=shuffle, after_batch=after_batch, num_workers=num_workers, **kwargs)

    def create_batch(self, b): return self.dataset.iloc[b]

TabularPandasIdentity._dl_type = TabDataLoaderIdentity

# Experiment Helpers 

In [ ]:
#export
def tabular_pandas(df, cat_names, cont_names, y_names, val_pct=0.2, tabular_type=TabularPandas):
    splits = RandomSplitter(valid_pct=val_pct)(range_of(df))
    to = tabular_type(df, procs=[Categorify, FillMissing,Normalize], cont_names=cont_names, cat_names=cat_names,
                           y_names=y_names, splits=splits, y_block=CategoryBlock())
    return to

In [ ]:
#export 
@delegates(TabNetClassifier)
def tabnet_df_classifier(df, cat_names, cont_names, y_names, val_pct, head=linear_head, cbs=[], enc=None, **kwargs):
    to = tabular_pandas(df, cat_names, cont_names, y_names, val_pct=val_pct)
    dls = to.dataloaders(bs=kwargs['bs'])
    model = TabNetClassifier(head, to, **kwargs)
    if enc is not None: model.enc = enc
    cbs=[SetPrior(), MaskRegularizer(kwargs['lambda_sparse']), *cbs]
    return Learner(dls, model, CrossEntropyLossFlat(), cbs=cbs, metrics=[accuracy])

In [ ]:
#export
@delegates(TabNetSelfSupervised)
def tabnet_df_self_sup(df, cat_names, cont_names, y_names, val_pct, head=tabnet_decoder, 
                       loss_func=MaskReconstructionLoss(), cbs=[], curriculum=False, p=0.8, **kwargs):
    to = tabular_pandas(df, cat_names, cont_names, y_names, tabular_type=TabularPandasIdentity, val_pct=val_pct)
    dls = to.dataloaders(bs=kwargs['bs'])
    dls.n_inp = 2
    cbs = [SetPrior(), TabularMasking(p=p, curriculum=curriculum), MaskRegularizer(kwargs['lambda_sparse']), *cbs]
    model = TabNetSelfSupervised(head, to, **kwargs)
    return Learner(dls, model, cbs=cbs, loss_func=loss_func, metrics=[mse])

In [ ]:
#export 
@delegates(tabnet_df_self_sup)
def score_before_after_ss(df, ds_params, val_pcts=[0.2]*3, cycle_lr=[(10, 1e-1/2)]*3, **kwargs):
    learn = tabnet_df_classifier(df, val_pct=val_pcts[0], **ds_params, **kwargs)
    learn.fit_one_cycle(*cycle_lr[0]) 
    before = learn.get_preds()
    
    learn_ss = tabnet_df_self_sup(df, val_pct=val_pcts[1], **ds_params, **kwargs)
    set_trace()
    learn_ss.fit_one_cycle(*cycle_lr[1])
    
    learn = tabnet_df_classifier(df, val_pct=val_pcts[2], **ds_params, enc=learn_ss.model.enc, **kwargs)
    learn.fit_one_cycle(*cycle_lr[2]) 
    after = learn.get_preds()

    return (before, after)

# Export

In [9]:
from nbdev.export import notebook2script
notebook2script()

Converted 01_core.ipynb.
Converted 02_model.ipynb.
Converted 03_experiments.ipynb.
Converted 04_utils.ipynb.
Converted index.ipynb.
